In [2]:
"""
Preamble for most code and jupyter notebooks
@author: bridgetsmart
@notebook date: 22 Aug 2023
"""

import numpy as np, pandas as pd

import matplotlib.pyplot as plt, seaborn as sns
import matplotlib as mpl

import math, string, re, pickle, json, time, os, sys, datetime, itertools

from tqdm.notebook import tqdm

%cd ..
from AUNewsPoliticsDatasetTwitter.load_data import load_data
from collections.abc import Iterable

# need a function to flatten irregular list of lists
def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

/Users/bridgetsmart/Documents/GitHub/AustralianInformationLandscape


## When I say 'sample data', I really mean the old data before I double checked all processing and added the extra tweets.

In [2]:
# load in existing data
# account classifications
account_classifications = load_data(selection=6, liwc_logical = False)

# HoR user details
HoRUsers = load_data(selection=5, liwc_logical = False)

# 2022 HoR Following
HoR2022Following = load_data(selection=4, liwc_logical = False)

# tweets
all_tweets = load_data(selection=1, liwc_logical = False)


Expect to wait around 7-8 minutes...


In [19]:
tweets = load_data()

In [21]:
tweets.columns

Index(['lang', 'text', 'tokenised_text', 'id', 'author_id', 'created_at',
       'mentions', 'urls', 'hashtags', 'referenced_tweets', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'preprocessed_text',
       'neg_sentiment', 'neutral_sentiment', 'positive_sentiment',
       'compound_sentiment', 'twitter_handle', 'account_type'],
      dtype='object')

In [2]:
%cd DashVisualisation

/Users/bridgetsmart/Documents/GitHub/AustralianInformationLandscape/DashVisualisation


In [3]:
# Let's add in something else
news_availability = pd.read_csv('/Users/bridgetsmart/Documents/GitHub/AustralianInformationLandscape/Data/Australian News Index (PIJI) Producers.csv')

In [4]:
lga = list(flatten([x.split(';') if type(x)==str else x for x in news_availability.lga_coverage.values]))
lga = list(set(lga))

In [5]:
len(list(set(lga)))

550

In [7]:
# Sources: https://www.aec.gov.au/profiles/files/2022/division-finder-sa.pdf (for each state)

# lga_federal_div = np.load("lga_federal_div.npy", allow_pickle=True)

with open('geospatialdata/lga_federal_div.json','r') as f:
    lga_federal_div = json.load(f)

In [ ]:
## explore out lga coverage

In [9]:
def get_fed_div(x):
    try:
        return list(set(flatten([lga_federal_div[y] for y in x.split(";")])))
    except:
        return x

In [10]:
news_availability['federal_division'] = news_availability.lga_coverage.apply(get_fed_div)

# setting up electorate data

In [16]:
electorate_data = pd.read_csv('/Users/bridgetsmart/Documents/GitHub/AustralianInformationLandscape/Data/commonwealth electorate data.csv')

# sample information flow data

In [4]:
net_flows = pd.read_csv('/Users/bridgetsmart/Documents/Python/Russian Trolls_Apr 2022/data/fivethirtyeight_Russia_bots_information_flow.csv')

In [5]:
net_flows

,source,target,net_information_flow,net_shannon_flow,forward_cross_entropy_rate,backward_cross_entropy_rate,forward_shannon_cross,backward_shannon_cross
0,VEROSANRRT,THEFOUNDINGSON,0.553476,0.017550,13.540496,7.805770,8.675281,8.657731
1,VEROSANRRT,USA_GUNSLINGER,0.551629,-0.296435,13.164866,7.410741,8.222540,8.518975
2,VEROSANRRT,KARENPARKER93,0.616116,0.488848,15.013628,7.601368,8.624982,8.136134
3,VEROSANRRT,ANNIEPOSHES,0.588456,0.157617,15.013628,7.944315,7.345372,7.187755
4,VEROSANRRT,CORNELLBURCHET,0.569696,0.675887,14.701180,7.793867,8.242399,7.566512
...,...,...,...,...,...,...,...,...
97898,SOUTHLONESTAR,PUREDAVIE,0.093185,0.368510,9.110186,7.892037,7.832971,7.464461
97899,SOUTHLONESTAR,JOTEFOKIS,0.239341,-1.050157,10.615711,6.818601,7.191708,8.241865
97900,SOUTHLONESTAR,J0HNLARSEN,0.065053,1.268940,8.294708,7.532066,9.636255,8.367315
97901,SOUTHLONESTAR,CLAYPAIGEBOO,0.045241,-0.724081,8.453541,7.801716,7.245596,7.969678


In [6]:
handles_hor = HoRUsers['Twitter Handle'].unique()

In [15]:
# create generic list of 150 handles
handles = [f'handle{i}' for i in range(150)]

In [17]:
# make a samlpe net flow matrix with the sources and targets as the handles with a random net flow drawn from a exponential distribution
choose_2 = list(itertools.combinations(handles, 2))

# create net flows
net_flows = pd.DataFrame(choose_2, columns = ['source', 'target'])

for comb in choose_2:
    net_flows.loc[(net_flows['source'] == comb[0]) & (net_flows['target'] == comb[1]), 'net_flow'] = np.random.exponential(1)

    # add a flow for each year
    for year in range(2014, 2022):
        net_flows.loc[(net_flows['source'] == comb[0]) & (net_flows['target'] == comb[1]), f'net_flow_{year}'] = np.random.exponential(0.6)

In [18]:
# normalise each column in the net flow matrix
for col in net_flows.columns[2:]:
    net_flows[col] = net_flows[col]/net_flows[col].max()
net_flows

,source,target,net_flow,net_flow_2014,net_flow_2015,net_flow_2016,net_flow_2017,net_flow_2018,net_flow_2019,net_flow_2020,net_flow_2021
0,handle0,handle1,0.004030,0.254405,0.042278,0.066539,0.104262,0.020424,0.172935,0.004668,0.003962
1,handle0,handle2,0.073348,0.045604,0.194916,0.157744,0.089185,0.007550,0.270217,0.038404,0.067318
2,handle0,handle3,0.080422,0.359101,0.018787,0.025379,0.197707,0.074810,0.019069,0.302948,0.000444
3,handle0,handle4,0.014711,0.357103,0.018587,0.160150,0.007569,0.013431,0.148887,0.024192,0.009210
4,handle0,handle5,0.064859,0.094407,0.156190,0.101169,0.001894,0.251460,0.047082,0.036849,0.019791
...,...,...,...,...,...,...,...,...,...,...,...
11170,handle146,handle148,0.007033,0.127126,0.050061,0.067045,0.198928,0.044668,0.112180,0.039769,0.067137
11171,handle146,handle149,0.007210,0.061277,0.029318,0.005813,0.077148,0.128250,0.053301,0.142267,0.192761
11172,handle147,handle148,0.116433,0.115941,0.015028,0.026669,0.362053,0.067057,0.099605,0.237422,0.129649
11173,handle147,handle149,0.002853,0.026115,0.080706,0.175232,0.131022,0.019728,0.086124,0.338677,0.009738


In [20]:
# add category for each source
categories = ['left', 'greens', 'oranges' 'right', 'centre']
category_dict = {handle: np.random.choice(categories) for handle in handles}

net_flows['source_category'] = net_flows['source'].apply(lambda x: category_dict[x])


In [21]:
net_flows['target_category'] = net_flows['target'].apply(lambda x: category_dict[x])   

In [23]:
# save the sample net flow matrix
net_flows.to_csv('sample_net_information_flows.csv', index = False)

In [ ]:
# This dataset is a sample of the net information flows between 150 handles. The net flow is a random number drawn from an exponential distribution. The net flow for each year is a random number drawn from an exponential distribution. The net flow for each year is normalised by the maximum net flow for that year.

In [3]:
# sample data between aggregated groups
categories = ['left','right', 'centre']
# make a samlpe net flow matrix with the sources and targets as the handles with a random net flow drawn from a exponential distribution
choose_2 = list(itertools.combinations(categories, 2))

# create net flows
net_flows = pd.DataFrame(choose_2, columns = ['source', 'target'])

for comb in choose_2:
    net_flows.loc[(net_flows['source'] == comb[0]) & (net_flows['target'] == comb[1]), 'net_flow'] = np.random.exponential(1)

    # add a flow for each year
    for year in range(2014, 2022):
        net_flows.loc[(net_flows['source'] == comb[0]) & (net_flows['target'] == comb[1]), f'net_flow_{year}'] = np.random.exponential(0.6)
# normalise each column in the net flow matrix
for col in net_flows.columns[2:]:
    net_flows[col] = net_flows[col]/net_flows[col].max()
net_flows

,source,target,net_flow,net_flow_2014,net_flow_2015,net_flow_2016,net_flow_2017,net_flow_2018,net_flow_2019,net_flow_2020,net_flow_2021
0,left,right,1.000000,0.004351,0.906585,1.000000,0.228546,0.264073,0.223116,0.268152,1.000000
1,left,centre,0.109211,1.000000,0.657953,0.491612,0.695320,1.000000,0.027330,0.419454,0.360901
2,right,centre,0.211056,0.071687,1.000000,0.286656,1.000000,0.708119,1.000000,1.000000,0.019804


In [4]:
# save the sample net flow matrix
net_flows.to_csv('sample_aggregated_net_information_flows.csv', index = False)

node_sizes_agg = {'category': categories, 'node_size': np.random.exponential(1, len(categories))}

In [5]:
import pickle
import numpy as np

In [6]:
node_sizes_agg = {c : 5*np.random.exponential() for c in categories}

In [7]:
node_sizes_agg

{'left': 0.8520963439389548,
 'right': 9.776799453633581,
 'centre': 8.329169086785729}

In [3]:
with open('supplementary_data/node_sizes.pkl', 'rb') as f:
    node_sizes = pickle.load(f)

In [9]:
with open('node_sizes_agg.pkl', 'wb') as f:
    pickle.dump(node_sizes_agg, f)